In [1]:
# Import Libraries
import numpy as np
import cv2

In [2]:
# Generating Function
def nerual_style_transfer(img, model, size = 30, upscale = 1):
    model_file_path = './models/'
    style = cv2.imread('art/' + str(model)[:-3] + '.jpg')

    # Load Neural Style Transfer Model
    neural_style_model = cv2.dnn.readNetFromTorch(model_file_path + model + '.t7')
    
    # Resize to fix height. You can change it as you want
    height, width = int(img.shape[0]), int(img.shape[1])
    new_width = int((size/height) * width)
    resized_image = cv2.resize(img,
                              (new_width, size),
                              interpolation = cv2.INTER_AREA)
    
    # Create our blob from the image
    # Perform a forward pass run of the network
    inp_blob = cv2.dnn.blobFromImage(resized_image,
                                     1.0,
                                    (new_width,640),
                                    (103.93, 116.77, 123.68),
                                    swapRB = False,
                                    crop = False)
    
    neural_style_model.setInput(inp_blob)
    output = neural_style_model.forward()
    
    # Reshape the output Tensor,
    # Add back the Mean Substruction
    # Re-order the channels
    output = output.reshape(3,output.shape[2],output.shape[3])
    output[0] += 103.93
    output[1] += 116.77
    output[2] += 123.68
    output /= 255
    output = output.transpose(1, 2, 0)
    output = cv2.resize(output,
                        None,
                        fx = upscale,
                        fy = upscale,
                        interpolation = cv2.INTER_LINEAR)
    return output


In [3]:
# Open the Video
# If you want to see yourself from your webcam
# replace the string inside the parenthesis
# with number 0
# it should be: VideoCapture(0)
cap = cv2.VideoCapture('./video/run.mp4')

In [4]:
# Our Style Transfers are:
# la_muse, feathers, composition_vii, mosaic, the_scream, udnie, starry_night, candy, the_wave
# While loop
while True:
    
    # Read the frame
    ret, frame = cap.read()
    
    # Display the output
    cv2.imshow('Nerual Style Transfer Video',
              nerual_style_transfer(frame, 'starry_night', 320, 2))
        
    # Close the frame
    if cv2.waitKey(1) & 0xFF ==27:
        break
    
# Release and Destroy
cap.release()
cv2.destroyAllWindows()

